In [ ]:
!pip install vadersentiment

In [ ]:
!pip install datasets

In [ ]:
!pip install transformers[torch]

# Preprocessing

In [ ]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Oil Prediction/Labeled.csv')

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2370 entries, 0 to 2369
Data columns (total 16 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   subreddit                        2253 non-null   object 
 1   selftext                         1371 non-null   object 
 2   author_fullname                  2157 non-null   object 
 3   title                            2178 non-null   object 
 4   Hate Level (High, Low,Not Hate)  2171 non-null   object 
 5   downs                            2177 non-null   object 
 6   upvote_ratio                     2177 non-null   object 
 7   ups                              2176 non-null   object 
 8   total_awards_received            2176 non-null   object 
 9   subreddit_type                   2176 non-null   object 
 10  created                          2176 non-null   object 
 11  all_awardings                    2176 non-null   object 
 12  awarders            

In [ ]:
dataset['Hate Level (High, Low,Not Hate)'].value_counts()

Hate Level (High, Low,Not Hate)
Not Hate    1647
Low          344
High         177
Hate           2
HIgh           1
Name: count, dtype: int64

In [ ]:
dataset = dataset[dataset['Hate Level (High, Low,Not Hate)'] != "Hate"]
dataset = dataset[dataset['Hate Level (High, Low,Not Hate)'] != "HIgh"]

In [ ]:
dataset_1 = dataset[['selftext', 'Hate Level (High, Low,Not Hate)']]

In [ ]:
dataset_1.dropna(subset=['selftext'], inplace=True)
dataset_1.dropna(subset=['Hate Level (High, Low,Not Hate)'], inplace=True)

In [ ]:
len(dataset_1)

1355

In [ ]:
dataset_1.head()

,selftext,"Hate Level (High, Low,Not Hate)"
0,Crude oli refinery for sale\nLocation in Unite...,Not Hate
2,This might be a silly question that's too comp...,Not Hate
5,Why are gas prices rising? Experts point to ex...,High
7,[Shell posts $9.6 billion profit even as energ...,Not Hate
9,Hello Everyone. I just recently partnered up w...,Low


In [ ]:
from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()
dataset_1['label'] = label_encoder.fit_transform(dataset_1['Hate Level (High, Low,Not Hate)'])


In [ ]:
dataset_1['label'].value_counts()

label
2    1052
1     211
0      92
Name: count, dtype: int64

In [ ]:
import pandas as pd
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import nltk

def clean_text(text):
    import re
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize
    nltk.download('stopwords')
    nltk.download('punkt')
    stop_words = set(stopwords.words('english'))
    text = re.sub(r'\W', ' ', str(text))
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    text = text.lower()
    text_tokens = word_tokenize(text)
    filtered_text = [word for word in text_tokens if word not in stop_words]
    return ' '.join(filtered_text)

dataset_1['cleaned_text'] = dataset_1['selftext'].apply(clean_text)



Streaming output truncated to the last 5000 lines.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading packa

In [ ]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    dataset_1['cleaned_text'], dataset_1['label'], test_size=0.2, random_state=42
)

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(texts):
    return tokenizer(texts, padding='max_length', truncation=True, max_length=512, return_tensors='pt')

train_encodings = tokenize_function(train_texts.tolist())
val_encodings = tokenize_function(val_texts.tolist())

# Sentiment Analysis

In [ ]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()


def analyze_sentiment(text):
    scores = analyzer.polarity_scores(text)
    return scores



In [ ]:
dataset_1['sentiment'] = dataset_1['selftext'].apply(lambda x: analyze_sentiment(x) if pd.notnull(x) else {'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0})

dataset_1['neg'] = dataset_1['sentiment'].apply(lambda x: x['neg'])
dataset_1['neu'] = dataset_1['sentiment'].apply(lambda x: x['neu'])
dataset_1['pos'] = dataset_1['sentiment'].apply(lambda x: x['pos'])
dataset_1['compound'] = dataset_1['sentiment'].apply(lambda x: x['compound'])

In [ ]:
dataset_1.drop(columns=['sentiment'], inplace=True)

In [ ]:
dataset_1.head()

,selftext,"Hate Level (High, Low,Not Hate)",label,cleaned_text,neg,neu,pos,compound
0,Crude oli refinery for sale\nLocation in Unite...,Not Hate,2,crude oli refinery sale location united arab e...,0.108,0.763,0.129,0.3676
2,This might be a silly question that's too comp...,Not Hate,2,might silly question complicated doubt great a...,0.058,0.862,0.081,0.2709
5,Why are gas prices rising? Experts point to ex...,High,0,gas prices rising experts point extreme heat o...,0.054,0.910,0.036,-0.3415
7,[Shell posts $9.6 billion profit even as energ...,Not Hate,2,shell posts 9 6 billion profit even energy pri...,0.029,0.795,0.176,0.9509
9,Hello Everyone. I just recently partnered up w...,Low,1,hello everyone recently partnered sellers agen...,0.041,0.840,0.120,0.8014


# Hate Speech Classification

In [ ]:
import torch

class RedditDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = RedditDataset(train_encodings, train_labels)
val_dataset = RedditDataset(val_encodings, val_labels)

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


TrainOutput(global_step=408, training_loss=0.6678312713024663, metrics={'train_runtime': 286.3297, 'train_samples_per_second': 11.358, 'train_steps_per_second': 1.425, 'total_flos': 855644834451456.0, 'train_loss': 0.6678312713024663, 'epoch': 3.0})

In [ ]:
trainer.evaluate()


{'eval_loss': 0.514904260635376,
 'eval_accuracy': 0.8118081180811808,
 'eval_runtime': 7.3261,
 'eval_samples_per_second': 36.991,
 'eval_steps_per_second': 4.641,
 'epoch': 3.0}

# Topic Modelling

In [ ]:
!pip install gensim nltk

In [ ]:
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess
from gensim.models import Phrases
from gensim.corpora import Dictionary
from gensim.models.phrases import Phraser

nltk.download('stopwords')
stop_words = stopwords.words('english')

def preprocess(texts):
    return [
        [word for word in simple_preprocess(str(doc)) if word not in stop_words]
        for doc in texts
    ]

processed_texts = preprocess(dataset['title'])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
bigram = Phrases(processed_texts, min_count=5, threshold=100)
trigram = Phrases(bigram[processed_texts], threshold=100)

bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

data_bigrams = make_bigrams(processed_texts)
data_trigrams = make_trigrams(data_bigrams)


In [ ]:
id2word = Dictionary(data_trigrams)
corpus = [id2word.doc2bow(text) for text in data_trigrams]

In [ ]:
from gensim.models.ldamodel import LdaModel

lda_model = LdaModel(corpus=corpus,
                     id2word=id2word,
                     num_topics=10,
                     random_state=100,
                     update_every=1,
                     chunksize=100,
                     passes=10,
                     alpha='auto',
                     per_word_topics=True)

for idx, topic in lda_model.print_topics(-1):
    print(f'Topic: {idx} \nWords: {topic}')


Topic: 0 
Words: 0.021*"years" + 0.019*"people" + 0.019*"help" + 0.017*"question" + 0.015*"soil" + 0.013*"plant" + 0.013*"go" + 0.010*"next" + 0.010*"russia" + 0.009*"current"
Topic: 1 
Words: 0.082*"nuclear" + 0.054*"energy" + 0.032*"power" + 0.024*"us" + 0.016*"looking" + 0.014*"get" + 0.014*"china" + 0.010*"used" + 0.010*"alberta" + 0.010*"canada"
Topic: 2 
Words: 0.092*"fuel" + 0.024*"new" + 0.019*"hydrogen" + 0.018*"nan" + 0.014*"eli" + 0.014*"one" + 0.012*"first" + 0.009*"may" + 0.009*"long" + 0.009*"fossil_fuels"
Topic: 3 
Words: 0.027*"heat" + 0.024*"french" + 0.019*"vs" + 0.015*"compared" + 0.013*"prototype" + 0.011*"deaths" + 0.011*"low" + 0.010*"group" + 0.008*"statistics" + 0.008*"film"
Topic: 4 
Words: 0.041*"water" + 0.024*"way" + 0.017*"development" + 0.015*"take" + 0.013*"true" + 0.012*"engineering" + 0.011*"production" + 0.011*"co" + 0.011*"cost" + 0.010*"leads"
Topic: 5 
Words: 0.033*"gold" + 0.018*"coal" + 0.017*"back" + 0.015*"company" + 0.015*"source" + 0.014*"proc